In [ ]:
# Import all necessary files.

from openpiv import tools, pyprocess, validation, filters, scaling 

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import imageio
import json

from os import listdir
from os.path import isfile, join


In [ ]:
# Initialize or read all the confiugration values

# Directory of the project
BASE_DIR = 'E:/flow/project/'

# Directory where video is kept
INPUT_DIR = BASE_DIR + ''

#Directory where images are stored
IMAGE_DIR = BASE_DIR + 'images/'

#Directory where substracted images are stored
SUB_DIR = BASE_DIR + 'subs/'

image_count = len([f for f in listdir(SUB_DIR) if isfile(join(SUB_DIR, f))]) - 2
print(image_count)



# Read the configuration file for threshold
config_file = open(BASE_DIR + 'config/config.json')
data = json.load(config_file)
thr_v = data["threshold_v"]
thr_u = data["threshold_u"]
print(thr_v)
print(thr_u)

In [ ]:

# Arrays for storing cumulative data.
master_x=None
master_y=None
master_u=None
master_v=None
master_s2n=None


# Read the first two frames
first_frame = tools.imread(BASE_DIR + 'subs/sub_000.jpg')
second_frame = tools.imread(BASE_DIR + 'subs/sub_001.jpg')


# Analysis Parameters to be passed into the algorithm
w_size = 32 # pixels, interrogation window size of first frame
s_size = 38  # pixels, interrogation window size of second frame
overlap = 12 # pixels, 50% overlap
dt = 0.02 # sec, time interval between successive frames.


u0, v0, sig2noise = pyprocess.extended_search_area_piv(first_frame.astype(np.int32), 
                                                           second_frame.astype(np.int32), 
                                                           window_size=w_size, 
                                                           overlap=overlap, 
                                                           dt=dt, 
                                                           search_area_size=s_size, 
                                                           sig2noise_method='peak2peak')

x, y = pyprocess.get_coordinates( image_size=first_frame.shape, 
                                     search_area_size=s_size, 
                                     overlap=overlap )

# Initialize the vectors from the first frame
master_x = x
master_y = y
master_u = u0
master_v = v0
master_s2n = sig2noise

# Run the analytics for rest of the images
for index in range(1,image_count):
    first_frame = tools.imread(SUB_DIR + 'sub_' + str(f"{index:003}")+'.jpg')
    second_frame = tools.imread(SUB_DIR + 'sub_' + str(f"{(index+1):003}")+'.jpg')
    print(SUB_DIR  + 'sub_'+str(f"{(index+1):003}")+'.jpg')

    
    u0, v0, sig2noise = pyprocess.extended_search_area_piv(first_frame.astype(np.int32), 
                                                           second_frame.astype(np.int32), 
                                                           window_size=w_size, 
                                                           overlap=overlap, 
                                                           dt=dt, 
                                                           search_area_size=s_size, 
                                                           sig2noise_method='peak2peak')
    
    x, y = pyprocess.get_coordinates( image_size=first_frame.shape, 
                                     search_area_size=s_size, 
                                     overlap=overlap )
    
    # Add vector for this frame to the consolidated vector
    master_x = np.concatenate((master_x,x))
    master_y = np.concatenate((master_y,y))
    master_u = np.concatenate((master_u,u0))
    master_v = np.concatenate((master_v,v0))
    master_s2n = np.concatenate((master_s2n,sig2noise))
    
# At this point of time, we have the conslidated vectors for all frames, not run the post processing on those vectors

u1, v1, mask = validation.sig2noise_val( master_u, master_v, 
                                        master_s2n, 
                                        threshold = 1.3 )


tools.save(master_x, master_y, u1, v1, mask, 'raw_vector.txt' )
    
# filter out outliers that are very different from the
# neighbours

u2, v2 = filters.replace_outliers( u1, v1, 
                                  method='localmean', 
                                  max_iter=3, 
                                  kernel_size=3)

# convert x,y to mm
# convert u,v to mm/sec

x, y, u3, v3 = scaling.uniform(master_x, master_y, u2, v2, 
                               scaling_factor = 96.52 ) # 96.52 microns/pixel

# 0,0 shall be bottom left, positive rotation rate is counterclockwise
x, y, u3, v3 = tools.transform_coordinates(x, y, u3, v3)

#save in the simple ASCII table format
tools.save(x, y, u3, v3, mask, 'processed_vector.txt' )


In [ ]:
plt.scatter(u3,v3)

In [ ]:
# Remove all negative and noise vectors
dest_file = open('filtered_vector.txt',"w")
u_vector = []
v_vector = []
x_val = []
y_val = []


with open('processed_vector.txt','r') as source_file:
    for line in source_file:
        #if ('1.0000' not in line and '-' not in line and 'x' not in line):
        if ('1.0000' not in line and  'x' not in line):
            dest_file.write(line)
            values = line.split()
            x_val.append(float(values[0]))
            y_val.append(float(values[1]))
            u_vector.append(float(values[2]))
            v_vector.append(float(values[3]))

dest_file.close()

average_u = sum(u_vector) / float(len(u_vector))
average_v = sum(v_vector) / float(len(v_vector))

print(average_u)
print(average_v)

dest_file = open('average_vector.txt',"w")

'''
for i in range(0, len(x_val)):
    if(v_vector[i] > 0):
        line = str(x_val[i]) + "    " + str(y_val[i]) + "    " + str(u_vector[i]) + "    " + str(v_vector[i]) + "    0.0000\n"
        dest_file.write(line)
'''

for i in range(0, len(x_val)):
    if(v_vector[i] > 0):
        line = str(x_val[i]) + "    " + str(y_val[i]) + "    " + str(u_vector[i]*average_u/abs(u_vector[i])) + "    " + str(v_vector[i]*average_v/abs(v_vector[i])) + "    0.0000\n"
        dest_file.write(line)


dest_file.close()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
tools.display_vector_field('processed_vector.txt', 
                           ax=ax, scaling_factor=96.52, 
                           scale=200, # scale defines here the arrow length
                           width=0.0035, # width is the thickness of the arrow
                           on_img=True, # overlay on the image
                           image_name=BASE_DIR + 'images/frame_000.jpg');


In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
tools.display_vector_field('filtered_vector.txt', 
                           ax=ax, scaling_factor=96.52, 
                           scale=100, # scale defines here the arrow length
                           width=0.004, # width is the thickness of the arrow
                           on_img=False, # overlay on the image
                           image_name=BASE_DIR + 'images/frame_000.jpg');

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
tools.display_vector_field('average_vector.txt', 
                           ax=ax, scaling_factor=96.52, 
                           scale=5, # scale defines here the arrow length
                           width=0.004, # width is the thickness of the arrow
                           on_img=False, # overlay on the image
                           image_name=BASE_DIR + 'images/frame_000.jpg');